## 5.5 顧客層に応じた販売戦略(クラスタリング、次元圧縮)


### 共通事前処理

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# 乱数の種
random_seed = 123

### オリジナルURL
Wholesale customers Data Set 

https://archive.ics.uci.edu/ml/datasets/wholesale+customers

### 5.5.4 データ読み込みからデータ確認まで

#### データ読み込み

In [ ]:
# データ読み込み
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases\
/00292/Wholesale%20customers%20data.csv'

df = pd.read_csv(url)

# 日本語に変更
columns = ['販売チャネル', '地域', '生鮮食品', '乳製品', '食料品', 
    '冷凍食品', '洗剤_紙製品', '総菜']
df.columns = columns

#### データ確認

In [ ]:
# データ内容確認
display(df.head())

# サイズ確認
print(df.shape)

In [ ]:
# 欠損値確認
print(df.isnull().sum())

In [ ]:
# 販売チャネルのコード値確認
print(df['販売チャネル'].value_counts())
print()

# 地域のコード値確認
print(df['地域'].value_counts())

#### 販売チャネルのコード値の意味
1.  Horeca(ホテル・レストラン・カフェ) 298 
2. Retail(小売り) 142 

#### 地域のコード値の意味
1.  Lisbon(リスボン) 77 
2. Oporto(オプルトまたはポルト) 47 
3. Other Region(その他) 316 



#### ヒストグラム表示

In [ ]:
# 販売チャネルと地域を落としたデータを作る
df2 = df.drop(['販売チャネル', '地域'], axis=1)

# 分析対象項目のヒストグラム表示
from pylab import rcParams
rcParams['figure.figsize'] = (8, 8)
df2.hist(bins=20)
plt.tight_layout() 
plt.show()

### 5.5.5 クラスタリングの実施

In [ ]:
# データ前処理とデータ分割は不要

# アルゴリズムの選択
from sklearn.cluster import KMeans

# グループ数を定義
clusters=4

# アルゴリズムの定義
algorithm = KMeans(n_clusters=clusters, 
    random_state=random_seed)

In [ ]:
# 学習、予測の実施
y_pred = algorithm.fit_predict(df2)

# 結果の一部確認
print(y_pred[:20])

### 5.5.6 クラスタリング結果の分析

#### グループ別平均値計算

In [ ]:
# グループごとの平均値計算
df_cluster = df2.groupby(y_pred).mean()
display(df_cluster)

#### グループ別グラフ表示

In [ ]:
# グループ別の棒グラフ表示
df_cluster.plot(kind='bar',stacked=True,
    figsize=(10, 6),colormap='jet')
plt.show()

#### 分析結果

* **0**: 生鮮食品中心
* **1**: 食料品中心
* **2**: 大量購入
* **3**: 少量購入

#### グループと販売チャネル・地域との関係

In [ ]:
# クラスタと、チャネル・地域の関係を調べる

# 販売チャネルと地域のみをdf3に抽出
df3 = df[['販売チャネル', '地域']]

# グラフの大きさ設定
rcParams['figure.figsize'] = (6,3)

# グループごとのグラフ表示
for i in range(clusters):
    fig, ax = plt.subplots()
    w = df3[y_pred==i]
    print(f'==== グループ{i} ====')
    w.hist(ax=ax)
    plt.tight_layout()
    plt.show()

#### 分析結果
グループ0(生鮮)とグループ3(少量)は、チャネル1(Horeca)と関連が深い。   
グループ1(食品)とグループ2(大量)は、チャネル2(Retail)と関連が深い。  
地域とグループの関連は時に見いだせない。

### 5.5.7 次元圧縮の実施

In [ ]:
# アルゴリズムの選択
from sklearn.decomposition import PCA

# モデル生成
# 散布図表示が目的のため2次元に圧縮
pca = PCA(n_components=2)

In [ ]:
# 学習・変換の実施
d2 = pca.fit_transform(df2)

# 結果の一部表示
print(d2[:5,:])

### 5.5.8 次元圧縮の活用方法

#### 散布図表示

In [ ]:
# グループごとに色分けし散布図表示

plt.figure(figsize=(8,8))
marks = ['.', 'x', '*', '+']
labels = ['生鮮', '食品', '大量', '少量']
colors = ['grey', 'lightblue', 'blue', 'black']
for i in range(clusters):
  plt.scatter(d2[y_pred==i][:,0], d2[y_pred==i][:,1], 
    marker=marks[i], label=labels[i], s=100, c=colors[i])
plt.legend(fontsize=14)
plt.show()

#### 例外値の調査

In [ ]:
# 生鮮グループの例外値を調べる
display(df[d2[:,0] > 100000])

【考察】  
インデックス181の顧客は、特に生鮮食品の購買量が多いと考えられる

In [ ]:
# 大量グループの例外値を調べる
display(df[d2[:,1] > 80000])

【考察】  
インデックス85の顧客は、特に食料品と洗剤・紙製品の購買量が多いと考えられる

In [ ]:
# 統計情報確認
display(df2.describe())